In [22]:
from urllib.parse import urlencode
from urllib.request import urlopen
from datetime import datetime
import itertools
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
from datetime import datetime, timedelta


In [23]:
def dateToStr(date):
    l = date.split('.')
    l[2]=l[2][2:]
    return ''.join(l)

def getFileName(ticker,period,startDate,endDate):
    return f'./data/{ticker}_{period}_{dateToStr(startDate)}_{dateToStr(endDate)}.txt'

def getData(url, fileName):
    print("Стучимся на Финам по ссылке: "+url)
    txt=urlopen(url).readlines()
    local_file = open(fileName, "w") #задаём файл, в который запишем котировки.
    for line in txt: #записываем свечи строку за строкой. 
        local_file.write(line.strip().decode( "windows-1251" )+'\n')	
    local_file.close()
    txt_to_csv(fileName, fileName[:-3] + 'csv')
    print(f"Готово. Проверьте файл {fileName}")

# def getFileName(ticker,period,startDate,endDate):
#     return f'./data/{ticker}_{period}_{dateToStr(startDate)}_{dateToStr(endDate)}.csv'

def txt_to_csv(input_file, output_file):
    with open(input_file, 'r') as txt_file:
        lines = txt_file.readlines()

    with open(output_file, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')
        for line in lines:
            values = line.strip().split(' ')
            csv_writer.writerow(values)

In [25]:
# tickers={'AFKS':19715, 'AFLT':29, 'AGRO':399716, 'ALRS':81820, 'CBOM':420694, 
#          'CHMF':16136, 'ENPG':929597, 'FEES':20509, 'FIVE':491944, 'GAZP':16842,
#          'GLTR':2134426, 'GMKN':795, 'HYDR':20266, 'IRAO':20516, 'LKOH':8,
#          'MAGN':16782, 'MGNT':17086,'MOEX':152798, 'MTSS':15523, 'NLMK':17046,
#          'NVTK':17370, 'OZON':2179435, 'PHOR':81114,'PIKK':18654,'PLZL':17123,
#          'POLY':175924, 'POSI':2907747, 'QIWI':181610,'ROSN':17273,'RTKM':7,
#          'RUAL':414279,'SBER':3,'SBERP':23,'SELG':81360, 'SGZH':2477992,
#          'SNGS':4,'SNGSP':13, 'TATN':825, 'TATNP':826, 'TCSG':913710,
#          'TRNFP':1012, 'UPRO':18584, 'VKCO':2901666, 'VTBR':19043, 'YNDX':388383}
tickers={'AFKS':19715, 'AFLT':29}
periods = {'daily': 8,}
startDate = "01.01.2014"
endDate = "01.01.2024"
market = 0
start_date = datetime.strptime(startDate, "%d.%m.%Y").date()
start_date_rev=datetime.strptime(startDate, '%d.%m.%Y').strftime('%Y%m%d')
end_date = datetime.strptime(endDate, "%d.%m.%Y").date()
end_date_rev=datetime.strptime(endDate, '%d.%m.%Y').strftime('%Y%m%d')

In [19]:
current_date = datetime.now()
day_before = current_date - timedelta(days=100)
startDate = day_before.strftime("%d.%m.%Y")
endDate = day_before.strftime("%d.%m.%Y")

In [20]:
start_date = datetime.strptime(startDate, "%d.%m.%Y").date()
start_date_rev=datetime.strptime(startDate, '%d.%m.%Y').strftime('%Y%m%d')
end_date = datetime.strptime(endDate, "%d.%m.%Y").date()
end_date_rev=datetime.strptime(endDate, '%d.%m.%Y').strftime('%Y%m%d')

In [26]:
for ticker in tickers:
    for period in periods:
        fileName = getFileName(ticker,period,startDate,endDate)
        params = urlencode([
                            ('market', market), #на каком рынке торгуется бумага
                            ('em', tickers[ticker]), #вытягиваем цифровой символ, который соответствует бумаге.
                            ('code', ticker), #тикер нашей акции
                            ('apply',0), #не нашёл что это значит. 
                            ('df', start_date.day), #Начальная дата, номер дня (1-31)
                            ('mf', start_date.month - 1), #Начальная дата, номер месяца (0-11)
                            ('yf', start_date.year), #Начальная дата, год
                            ('from', start_date), #Начальная дата полностью
                            ('dt', end_date.day), #Конечная дата, номер дня	
                            ('mt', end_date.month - 1), #Конечная дата, номер месяца
                            ('yt', end_date.year), #Конечная дата, год
                            ('to', end_date), #Конечная дата
                            ('p', periods[period]), #Таймфрейм
                            ('f', ticker+"_" + start_date_rev + "_" + end_date_rev), #Имя сформированного файла
                            ('e', ".csv"), #Расширение сформированного файла
                            ('cn', ticker), #ещё раз тикер акции	
                            ('dtf', 1), #В каком формате брать даты. Выбор из 5 возможных. См. страницу https://www.finam.ru/profile/moex-akcii/sberbank/export/
                            ('tmf', 1), #В каком формате брать время. Выбор из 4 возможных.
                            ('MSOR', 0), #Время свечи (0 - open; 1 - close)	
                            ('mstime', "on"), #Московское время	
                            ('mstimever', 1), #Коррекция часового пояса	
                            ('sep', 1), #Разделитель полей	(1 - запятая, 2 - точка, 3 - точка с запятой, 4 - табуляция, 5 - пробел)
                            ('sep2', 1), #Разделитель разрядов
                            ('datf', 1), #Формат записи в файл. Выбор из 6 возможных.
                            ('at', 1)]) #Нужны ли заголовки столбцов
        FINAM_URL = "http://export.finam.ru/"
        url = FINAM_URL + ticker+"_" + start_date_rev + "_" + end_date_rev + ".csv?" + params
        getData(url, fileName)

Стучимся на Финам по ссылке: http://export.finam.ru/AFKS_20140101_20240101.csv?market=0&em=19715&code=AFKS&apply=0&df=1&mf=0&yf=2014&from=2014-01-01&dt=1&mt=0&yt=2024&to=2024-01-01&p=8&f=AFKS_20140101_20240101&e=.csv&cn=AFKS&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=1&sep2=1&datf=1&at=1
+++++++
Готово. Проверьте файл ./data/AFKS_daily_010114_010124.txt
Стучимся на Финам по ссылке: http://export.finam.ru/AFLT_20140101_20240101.csv?market=0&em=29&code=AFLT&apply=0&df=1&mf=0&yf=2014&from=2014-01-01&dt=1&mt=0&yt=2024&to=2024-01-01&p=8&f=AFLT_20140101_20240101&e=.csv&cn=AFLT&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=1&sep2=1&datf=1&at=1
+++++++
Готово. Проверьте файл ./data/AFLT_daily_010114_010124.txt


In [6]:
url

'http://export.finam.ru/YNDX_20140101_20240101.csv?market=0&em=388383&code=YNDX&apply=0&df=1&mf=0&yf=2014&from=2014-01-01&dt=1&mt=0&yt=2024&to=2024-01-01&p=8&f=YNDX_20140101_20240101&e=.csv&cn=YNDX&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=1&sep2=1&datf=1&at=1'